In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options 
from selenium.webdriver.common.keys import Keys
import chromedriver_binary
import gspread
import time
import cv2
import numpy as np
from PIL import Image
import requests

from oauth2client.service_account import ServiceAccountCredentials 
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name('suraping-381608-4562bacb42ee.json', scope)

def download_img(image, filename):
    r = requests.get(url=image, cookies=cookie, headers=headers, stream=True)
    if r.status_code == 200:
        with open(filename, 'wb') as f:
            f.write(r.content)

date = 1 #当日は0、前日は1・・・
date2 = "08/18"
pmc = "15008030" # 店舗ID
SPREADSHEET_KEY = '1PG-sk-FpaxIWhexavaj2Jakkxb7vUd5CBfQLMtpnsT8' # スプレッドシートのID
pstart = 1 #1 パチンコの1台目
sstart = 1071 #1071 スロットの1台目
wait = 0.01

user_id = "tomosvtg.1988+456@gmail.com"
user_pw = "paruparu22"

Template_p_path = "site777_2/template_p.png"
Template_s_path = "site777_2/template_s.png"
dot_point_p = 130000 / 432
dot_point_px = 60000 / 234
dot_point_s = 20000 / 432

mobile_emulation = { "deviceName": "iPhone 12 Pro" }

headers = {
    "User-Agent": "Mozilla/5.0 (iPhone; CPU iPhone OS 16_1 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) CriOS/108.0.5359.112 Mobile/15E148 Safari/604.1",
    "Referer" : "https://m.site777.jp/f/D3000.do?pmc=22021007&gc=2&dn=5019&urt=2173&mdc=120104&dsgk=0&dtdd=2"
}

options = Options()
options.add_experimental_option("mobileEmulation", mobile_emulation)
#options.add_argument('--headless')
options.binary_location = 'C:\\Program Files\\Google\\Chrome\\Application\\Chrome.exe'
chrome_service = ChromeService(executable_path='C:\\Users\\garden026\\WD\\114.0.5735.90\\chromedriver.exe')

gc = gspread.authorize(credentials)
wb = gc.open_by_key(SPREADSHEET_KEY)

worksheet = wb.worksheet("Config")
p4row = worksheet.cell(11,  23).value
p1row = worksheet.cell(11,  24).value
p2row = worksheet.cell(11,  25).value
s20row = worksheet.cell(11,  28).value
s5row = worksheet.cell(11,  29).value

browser = webdriver.Chrome(service=chrome_service,options=options)
browser.command_executor._commands["send_command"] = ("POST", '/session/$sessionId/chromium/send_command')
params = {'cmd': 'Page.setDownloadBehavior', 'params': {'behavior': 'allow', 'downloadPath': 'site777'}}
command_result = browser.execute("send_command", params)

browser.get('https://m.site777.jp/f/A0100.do')
time.sleep(1)

browser.find_element("xpath",'/html/body/section[1]/div/div/p[3]/a').click()
time.sleep(1)

browser.find_element("xpath",'/html/body/section/section[2]/div/div/p').click()
time.sleep(1)

browser.find_element("xpath",'/html/body/div[1]/div/section/div/dl/dd/form/div[1]/input').send_keys(user_id)
browser.find_element("xpath",'/html/body/div[1]/div/section/div/dl/dd/form/div[2]/input').send_keys(user_pw)
time.sleep(1)

browser.execute_script('document.fmLogin.submit()')
time.sleep(1)

browser.get('https://m.site777.jp/f/D0400.do?pmc=' + str(pmc))
time.sleep(1)

browser.find_element("xpath",'/html/body/div[2]/section/div[1]/dl/dd/form/input[1]').send_keys(pstart)
time.sleep(1)

browser.execute_script("S7Recaptcha.grv3FormSubmit(S7Recaptcha.GRV3_ACTION_HOMEPAGE,document.forms['formdaino'])")
time.sleep(1)

pmdc = browser.current_url.split("mdc=")[1]

browser.get('https://m.site777.jp/f/D0400.do?pmc=' + str(pmc))
time.sleep(1)

browser.find_element("xpath",'/html/body/div[2]/section/div[1]/dl/dd/form/input[1]').send_keys(sstart)
time.sleep(1)

browser.execute_script("S7Recaptcha.grv3FormSubmit(S7Recaptcha.GRV3_ACTION_HOMEPAGE,document.forms['formdaino'])")
time.sleep(1)

smdc = browser.current_url.split("mdc=")[1]

ck = browser.get_cookies()
k = 0
while k < 30:
    ccheck = ck[k]["name"]
    if ccheck == "PO_SP_ID":
        po_sp_id = ck[k]["value"]
        k = 30
    k = k + 1
k = 0
while k < 30:
    ccheck = ck[k]["name"]
    if ccheck == "JSESSIONID":
        jsessionid = ck[k]["value"]
        k = 30
    k = k + 1

cookie = {"PO_SP_ID" : po_sp_id,
         "JSESSIONID" : jsessionid}

browser.close()

p4table = []
p1table = []
p2table = []
s20table = []
s5table = []

# Pachinko

dn = pstart
mdc = pmdc

roop = 0
while roop < 1:
    try:
        url = "https://m.site777.jp/f/D2600S.do?pmc=" + str(pmc) + "&gc=2&dtdd=" + str(date) + "&dsgk=0&dn=" + str(dn) + "&mdc=" + str(mdc)
        response = requests.get(url=url, headers=headers, cookies=cookie)
        jsonData = response.text

        i = 1
        while i < 100:
            try:
                temp = jsonData.split("<daiNo>")[i].split("</daiNo>")[0]
                if int(dn) == int(temp):
                    m = i
                    i = 1000
                i = i + 1
            except:
                i = 1000
                continue
            else:
                pass

        cc = 0
        while cc < 30:
            try:
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                unit = jsonData.split("<unit>")[m + cc].split("</unit>")[0]
                bb = jsonData.split("<bonusCount>")[m + cc].split("</bonusCount>")[0]
                rb = jsonData.split("<hatariSu>")[m + cc].split("</hatariSu>")[0]

                image = "https://m.site777.jp" + jsonData.split("<bigGraphUrl>")[m + cc].split("</bigGraphUrl>")[0].replace("amp;","")

                k = 0
                while(k < 10):
                    try:
                        filename =  'site777_2/dai_'+str(dn)+'.png'
                        download_img(image, filename)

                        time.sleep(wait)

                        Target_path = filename

                        template = cv2.imread(Template_p_path)
                        template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)

                        img_rgb = cv2.imread(Target_path)
                        img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)

                        res = cv2.matchTemplate(img_gray,template_gray,cv2.TM_CCOEFF_NORMED)

                        threshold = 0.6
                        loc = np.where( res >= threshold)

                        Mergin = 5
                        index = []
                        for iii in range(len(loc[1][:])):
                            count = 0
                            for j in range(len(loc[1][:])):
                                if abs(loc[1][:][iii] - loc[1][:][j]) < Mergin and abs(loc[0][:][iii] - loc[0][:][j]) < Mergin:
                                    if count > 0:
                                        index.append(j)
                                    count = count + 1

                        loc = np.delete(loc, index, axis=1)

                        w, h = template_gray.shape[::-1]

                        cv2.imwrite("site777_2/template_result.png",img_rgb)

                        img_bin = cv2.Canny(img_gray,128,200)
                        cv2.imwrite("site777_2/bin_result.png",img_bin)

                        count = 0

                        for x in range(0,234):
                            xx = 238 - x
                            a = 9
                            b = 442
                            if xx < 53:
                                a = 248
                                b = 435
                            for y in range(a,b):
                                if y < 341 or y > 343:
                                    if all(img_bin[y, xx]==[255, 255, 255]):
                                        point = 342 - y
                                        count = count + 1
                                        break
                            if count == 1:
                                break

                        game = round(dot_point_px * ( xx - 10) , 0)
                        samai = round(dot_point_p * point ,0)-300
                        if xx==10:
                            samai = 0
                            game = 0

                        k = 10

                    except:
                        k = k + 1
                        continue

                    else:
                        pass
                    
                temp = jsonData.split("<daiNo>")[m + cc + 1].split("</daiNo>")[0]
                li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb)).split("|")
                print(li)
                if unit == "400":
                    p4table.append(li)
                if unit == "112":
                    p1table.append(li)
                if unit == "20":
                    p2table.append(li)
                cc = cc + 1

            except:
                if cc == 0:
                    li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb)).split("|")
                    print(li)
                    if unit == "400":
                        p4table.append(li)
                    if unit == "112":
                        p1table.append(li)
                    if unit == "20":
                        p2table.append(li)
                    roop = 1
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                cc = 30
                continue
            else:
                pass
            
    except:
        continue
    else:
        pass

# Slot

dn = sstart
mdc = smdc

roop = 0
while roop < 1:
    try:
        url = "https://m.site777.jp/f/D3000S.do?pmc=" + str(pmc) + "&gc=2&dtdd=" + str(date) + "&dsgk=0&dn=" + str(dn) + "&mdc=" + str(mdc)
        response = requests.get(url=url, headers=headers, cookies=cookie)
        jsonData = response.text

        i = 1
        while i < 1000:
            try:
                temp = jsonData.split("<daiNo>")[i].split("</daiNo>")[0]
                if int(dn) == int(temp):
                    m = i
                    i = 1000
                i = i + 1
            except:
                i = 1000
                continue
            else:
                pass

        cc = 0
        while cc < 30:
            try:
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                unit = jsonData.split("<unit>")[m + cc].split("</unit>")[0]
                game = jsonData.split("<totalGameCount>")[m + cc].split("</totalGameCount>")[0]
                bb = jsonData.split("<bonusCount>")[m + cc].split("</bonusCount>")[0]
                rb = jsonData.split("<regularCount>")[m + cc].split("</regularCount>")[0]
                art = jsonData.split("<artCount>")[m + cc].split("</artCount>")[0]

                image = "https://m.site777.jp" + jsonData.split("<bigGraphUrl>")[m + cc].split("</bigGraphUrl>")[0].replace("amp;","")

                k = 0
                while(k < 10):
                    try:
                        filename =  'site777_2/dai_'+str(dn)+'.png'
                        download_img(image, filename)

                        time.sleep(wait)

                        Target_path = filename

                        template = cv2.imread(Template_s_path)
                        template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)

                        img_rgb = cv2.imread(Target_path)
                        img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)

                        res = cv2.matchTemplate(img_gray,template_gray,cv2.TM_CCOEFF_NORMED)

                        threshold = 0.6
                        loc = np.where( res >= threshold)

                        Mergin = 5
                        index = []
                        for iii in range(len(loc[1][:])):
                            count = 0
                            for j in range(len(loc[1][:])):
                                if abs(loc[1][:][iii] - loc[1][:][j]) < Mergin and abs(loc[0][:][iii] - loc[0][:][j]) < Mergin:
                                    if count > 0:
                                        index.append(j)
                                    count = count + 1

                        loc = np.delete(loc, index, axis=1)

                        w, h = template_gray.shape[::-1]

                        cv2.imwrite("site777_2/template_result.png",img_rgb)

                        img_bin = cv2.Canny(img_gray,128,200)
                        cv2.imwrite("site777_2/bin_result.png",img_bin)

                        count = 0

                        for x in range(0,234):
                            xx = 238 - x
                            a = 9
                            b = 442
                            if xx < 53:
                                a = 248
                                b = 435
                            for y in range(a,b):
                                if y < 332 or y > 334:
                                    if all(img_bin[y, xx]==[255, 255, 255]):
                                        point = 333 - y
                                        count = count + 1
                                        break
                            if count == 1:
                                break

                        samai = round(dot_point_s * point ,0)
                        if xx==10:
                            samai = 0

                        k = 10

                    except:
                        k = k + 1
                        continue

                    else:
                        pass

                temp = jsonData.split("<daiNo>")[m + cc + 1].split("</daiNo>")[0]
                li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb) + "|" + str(art)).split("|")
                print(li)
                if unit == "2173":
                    s20table.append(li)
                if unit == "561":
                    s5table.append(li)
                cc = cc + 1

            except:
                if cc == 0:
                    li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb) + "|" + str(art)).split("|")
                    print(li)
                    if unit == "2173":
                        s20table.append(li)
                    if unit == "561":
                        s5table.append(li)
                    roop = 1
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                cc = 30
                continue
            else:
                pass
            
    except:
        continue
    else:
        pass

wb.values_append("'P4RAW'!F" + str(int(p4row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": p4table})
wb.values_append("'P1RAW'!F" + str(int(p1row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": p1table})
wb.values_append("'P0.2RAW'!F" + str(int(p2row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": p2table})
wb.values_append("'S20RAW'!F" + str(int(s20row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": s20table})
wb.values_append("'S5RAW'!F" + str(int(s5row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": s5table})

['08/18', '025740', '1', '26667.0', '25580.0', '82', '3']
['08/18', '025667', '2', '48974.0', '21969.0', '44', '11']
['08/18', '025572', '3', '44359.0', '-4212.0', '36', '14']
['08/18', '025107', '5', '35128.0', '-4814.0', '41', '9']
['08/18', '025593', '6', '30769.0', '-12939.0', '11', '4']
['08/18', '025777', '7', '40256.0', '2107.0', '54', '22']
['08/18', '025850', '8', '37692.0', '-20462.0', '8', '4']
['08/18', '025573', '10', '46154.0', '-15948.0', '23', '14']
['08/18', '025647', '11', '54359.0', '-3911.0', '78', '27']
['08/18', '025606', '12', '0', '0', '0', '0']
['08/18', '025787', '13', '31026.0', '2408.0', '14', '10']
['08/18', '025838', '15', '36154.0', '-29490.0', '2', '2']
['08/18', '025579', '16', '45897.0', '12339.0', '27', '5']
['08/18', '025603', '17', '34103.0', '2709.0', '34', '16']
['08/18', '025521', '18', '51795.0', '6320.0', '62', '24']
['08/18', '025521', '20', '42564.0', '-12036.0', '29', '16']
['08/18', '025521', '21', '47692.0', '-4814.0', '37', '18']
['08/18'

['08/18', '025844', '217', '3846.0', '-3610.0', '0', '0']
['08/18', '025844', '218', '3590.0', '-2406.0', '2', '2']
['08/18', '025844', '220', '9231.0', '-7522.0', '3', '3']
['08/18', '025847', '221', '3333.0', '-1504.0', '9', '2']
['08/18', '025847', '222', '11795.0', '1205.0', '44', '5']
['08/18', '025847', '223', '7179.0', '-5416.0', '5', '2']
['08/18', '025847', '225', '8718.0', '-3610.0', '14', '5']
['08/18', '025885', '226', '1026.0', '-902.0', '1', '1']
['08/18', '025868', '227', '2564.0', '-902.0', '5', '1']
['08/18', '025563', '228', '9744.0', '904.0', '12', '3']
['08/18', '025866', '230', '6923.0', '-4212.0', '6', '3']
['08/18', '025892', '231', '22308.0', '-2406.0', '15', '6']
['08/18', '025865', '232', '18205.0', '302.0', '50', '7']
['08/18', '025404', '233', '10769.0', '904.0', '12', '2']
['08/18', '025835', '235', '8974.0', '-902.0', '30', '5']
['08/18', '025575', '236', '21538.0', '4214.0', '38', '13']
['08/18', '025848', '237', '4359.0', '-902.0', '6', '2']
['08/18', '0

['08/18', '025667', '533', '17179.0', '6621.0', '14', '3']
['08/18', '025637', '535', '43846.0', '-1203.0', '46', '26']
['08/18', '025392', '536', '15385.0', '-5115.0', '22', '6']
['08/18', '025523', '537', '33846.0', '-3911.0', '36', '12']
['08/18', '025871', '538', '12051.0', '23172.0', '22', '3']
['08/18', '025525', '550', '23333.0', '-10231.0', '5', '3']
['08/18', '025683', '551', '16154.0', '4816.0', '33', '8']
['08/18', '025773', '552', '17949.0', '16552.0', '23', '2']
['08/18', '025802', '553', '20256.0', '-8124.0', '17', '6']
['08/18', '025533', '555', '18205.0', '6019.0', '35', '8']
['08/18', '025647', '556', '31026.0', '603.0', '49', '15']
['08/18', '025826', '557', '10256.0', '-2707.0', '4', '3']
['08/18', '025856', '558', '31282.0', '-1805.0', '17', '8']
['08/18', '025659', '560', '26923.0', '-902.0', '31', '9']
['08/18', '025651', '561', '21538.0', '302.0', '21', '9']
['08/18', '025824', '562', '30256.0', '14445.0', '54', '6']
['08/18', '025791', '563', '22308.0', '-8425.0

['08/18', '025856', '758', '26154.0', '-9027.0', '9', '4']
['08/18', '025818', '760', '1026.0', '-902.0', '0', '0']
['08/18', '025867', '761', '25128.0', '-12939.0', '5', '5']
['08/18', '025835', '762', '35897.0', '-9629.0', '89', '24']
['08/18', '025831', '763', '11795.0', '-902.0', '14', '7']
['08/18', '025687', '765', '28462.0', '-2106.0', '16', '4']
['08/18', '025871', '766', '24359.0', '-7823.0', '11', '6']
['08/18', '025718', '767', '11795.0', '1506.0', '58', '3']
['08/18', '025667', '768', '18974.0', '12941.0', '20', '5']
['08/18', '025544', '770', '12308.0', '-3309.0', '11', '5']
['08/18', '025615', '771', '5897.0', '5418.0', '24', '2']
['08/18', '025838', '772', '8205.0', '-4212.0', '2', '1']
['08/18', '025285', '773', '35897.0', '-12337.0', '20', '10']
['08/18', '025523', '775', '25641.0', '9631.0', '41', '10']
['08/18', '025483', '776', '0', '0', '0', '0']
['08/18', '025790', '777', '14615.0', '-6920.0', '18', '10']
['08/18', '025501', '778', '22308.0', '4214.0', '43', '12']

['08/18', '120106', '1188', '1171', '-880.0', '3', '2', '--']
['08/18', '120106', '1200', '618', '-93.0', '4', '0', '--']
['08/18', '120103', '1201', '2238', '972.0', '15', '1', '7']
['08/18', '120103', '1202', '985', '417.0', '10', '2', '3']
['08/18', '120103', '1203', '166', '-139.0', '0', '0', '1']
['08/18', '120103', '1205', '1737', '3009.0', '40', '2', '6']
['08/18', '120103', '1206', '69', '0', '0', '0', '--']
['08/18', '120128', '1207', '2638', '-694.0', '0', '0', '7']
['08/18', '120128', '1208', '2889', '-833.0', '0', '0', '7']
['08/18', '120128', '1210', '0', '0', '0', '0', '--']
['08/18', '120128', '1211', '1808', '-1852.0', '0', '0', '2']
['08/18', '120128', '1212', '1726', '463.0', '0', '0', '7']
['08/18', '120110', '1213', '2002', '-1111.0', '7', '4', '--']
['08/18', '120110', '1215', '1077', '1620.0', '15', '1', '--']
['08/18', '120110', '1216', '3844', '2778.0', '36', '13', '--']
['08/18', '120110', '1217', '2368', '-463.0', '12', '7', '--']
['08/18', '120110', '1218', '

['08/18', '120015', '1505', '3397', '-93.0', '0', '12', '38']
['08/18', '120043', '1506', '1290', '-139.0', '4', '6', '--']
['08/18', '120026', '1507', '5810', '-417.0', '19', '17', '--']
['08/18', '120026', '1508', '6206', '2454.0', '32', '15', '--']
['08/18', '120026', '1510', '6459', '1898.0', '28', '22', '--']
['08/18', '120026', '1511', '3687', '370.0', '15', '9', '--']
['08/18', '120026', '1512', '6637', '1065.0', '27', '19', '--']
['08/18', '120026', '1513', '5819', '1667.0', '25', '20', '--']
['08/18', '120026', '1515', '6451', '4676.0', '40', '17', '--']
['08/18', '120010', '1516', '7475', '1019.0', '32', '25', '--']
['08/18', '120010', '1517', '2871', '-509.0', '8', '9', '--']
['08/18', '120010', '1518', '7471', '2963.0', '37', '29', '--']
['08/18', '120010', '1520', '3488', '-833.0', '10', '12', '--']
['08/18', '120010', '1521', '6989', '324.0', '27', '22', '--']
['08/18', '120010', '1522', '5226', '-1944.0', '11', '13', '--']
['08/18', '120010', '1523', '4545', '-93.0', '16

{'spreadsheetId': '1PG-sk-FpaxIWhexavaj2Jakkxb7vUd5CBfQLMtpnsT8',
 'updates': {'spreadsheetId': '1PG-sk-FpaxIWhexavaj2Jakkxb7vUd5CBfQLMtpnsT8',
  'updatedRange': 'S5RAW!F6760:M6799',
  'updatedRows': 40,
  'updatedColumns': 8,
  'updatedCells': 320}}

In [2]:


browser.find_element("xpath",'/html/body/section/section[2]/div/div/p').click()
time.sleep(1)

browser.find_element("xpath",'/html/body/div[1]/div/section/div/dl/dd/form/div[1]/input').send_keys(user_id)
browser.find_element("xpath",'/html/body/div[1]/div/section/div/dl/dd/form/div[2]/input').send_keys(user_pw)
time.sleep(1)

browser.execute_script('document.fmLogin.submit()')
time.sleep(1)

browser.get('https://m.site777.jp/f/D0400.do?pmc=' + str(pmc))
time.sleep(1)

browser.find_element("xpath",'/html/body/div[2]/section/div[1]/dl/dd/form/input[1]').send_keys(pstart)
time.sleep(1)

browser.execute_script("S7Recaptcha.grv3FormSubmit(S7Recaptcha.GRV3_ACTION_HOMEPAGE,document.forms['formdaino'])")
time.sleep(1)

pmdc = browser.current_url.split("mdc=")[1]

browser.get('https://m.site777.jp/f/D0400.do?pmc=' + str(pmc))
time.sleep(1)

browser.find_element("xpath",'/html/body/div[2]/section/div[1]/dl/dd/form/input[1]').send_keys(sstart)
time.sleep(1)

browser.execute_script("S7Recaptcha.grv3FormSubmit(S7Recaptcha.GRV3_ACTION_HOMEPAGE,document.forms['formdaino'])")
time.sleep(1)

smdc = browser.current_url.split("mdc=")[1]

ck = browser.get_cookies()
k = 0
while k < 30:
    ccheck = ck[k]["name"]
    if ccheck == "PO_SP_ID":
        po_sp_id = ck[k]["value"]
        k = 30
    k = k + 1
k = 0
while k < 30:
    ccheck = ck[k]["name"]
    if ccheck == "JSESSIONID":
        jsessionid = ck[k]["value"]
        k = 30
    k = k + 1

cookie = {"PO_SP_ID" : po_sp_id,
         "JSESSIONID" : jsessionid}

browser.close()

p4table = []
p1table = []
p2table = []
s20table = []
s5table = []

# Pachinko

dn = pstart
mdc = pmdc

roop = 0
while roop < 1:
    try:
        url = "https://m.site777.jp/f/D2600S.do?pmc=" + str(pmc) + "&gc=2&dtdd=" + str(date) + "&dsgk=0&dn=" + str(dn) + "&mdc=" + str(mdc)
        response = requests.get(url=url, headers=headers, cookies=cookie)
        jsonData = response.text

        i = 1
        while i < 100:
            try:
                temp = jsonData.split("<daiNo>")[i].split("</daiNo>")[0]
                if int(dn) == int(temp):
                    m = i
                    i = 1000
                i = i + 1
            except:
                i = 1000
                continue
            else:
                pass

        cc = 0
        while cc < 30:
            try:
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                unit = jsonData.split("<unit>")[m + cc].split("</unit>")[0]
                bb = jsonData.split("<bonusCount>")[m + cc].split("</bonusCount>")[0]
                rb = jsonData.split("<hatariSu>")[m + cc].split("</hatariSu>")[0]

                image = "https://m.site777.jp" + jsonData.split("<bigGraphUrl>")[m + cc].split("</bigGraphUrl>")[0].replace("amp;","")

                k = 0
                while(k < 10):
                    try:
                        filename =  'site777_2/dai_'+str(dn)+'.png'
                        download_img(image, filename)

                        time.sleep(wait)

                        Target_path = filename

                        template = cv2.imread(Template_p_path)
                        template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)

                        img_rgb = cv2.imread(Target_path)
                        img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)

                        res = cv2.matchTemplate(img_gray,template_gray,cv2.TM_CCOEFF_NORMED)

                        threshold = 0.6
                        loc = np.where( res >= threshold)

                        Mergin = 5
                        index = []
                        for iii in range(len(loc[1][:])):
                            count = 0
                            for j in range(len(loc[1][:])):
                                if abs(loc[1][:][iii] - loc[1][:][j]) < Mergin and abs(loc[0][:][iii] - loc[0][:][j]) < Mergin:
                                    if count > 0:
                                        index.append(j)
                                    count = count + 1

                        loc = np.delete(loc, index, axis=1)

                        w, h = template_gray.shape[::-1]

                        cv2.imwrite("site777_2/template_result.png",img_rgb)

                        img_bin = cv2.Canny(img_gray,128,200)
                        cv2.imwrite("site777_2/bin_result.png",img_bin)

                        count = 0

                        for x in range(0,234):
                            xx = 238 - x
                            a = 9
                            b = 442
                            if xx < 53:
                                a = 248
                                b = 435
                            for y in range(a,b):
                                if y < 341 or y > 343:
                                    if all(img_bin[y, xx]==[255, 255, 255]):
                                        point = 342 - y
                                        count = count + 1
                                        break
                            if count == 1:
                                break

                        game = round(dot_point_px * ( xx - 10) , 0)
                        samai = round(dot_point_p * point ,0)-300
                        if xx==10:
                            samai = 0
                            game = 0

                        k = 10

                    except:
                        k = k + 1
                        continue

                    else:
                        pass
                    
                temp = jsonData.split("<daiNo>")[m + cc + 1].split("</daiNo>")[0]
                li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb)).split("|")
                print(li)
                if unit == "400":
                    p4table.append(li)
                if unit == "112":
                    p1table.append(li)
                if unit == "20":
                    p2table.append(li)
                cc = cc + 1

            except:
                if cc == 0:
                    li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb)).split("|")
                    print(li)
                    if unit == "400":
                        p4table.append(li)
                    if unit == "112":
                        p1table.append(li)
                    if unit == "20":
                        p2table.append(li)
                    roop = 1
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                cc = 30
                continue
            else:
                pass
            
    except:
        continue
    else:
        pass

# Slot

dn = sstart
mdc = smdc

roop = 0
while roop < 1:
    try:
        url = "https://m.site777.jp/f/D3000S.do?pmc=" + str(pmc) + "&gc=2&dtdd=" + str(date) + "&dsgk=0&dn=" + str(dn) + "&mdc=" + str(mdc)
        response = requests.get(url=url, headers=headers, cookies=cookie)
        jsonData = response.text

        i = 1
        while i < 1000:
            try:
                temp = jsonData.split("<daiNo>")[i].split("</daiNo>")[0]
                if int(dn) == int(temp):
                    m = i
                    i = 1000
                i = i + 1
            except:
                i = 1000
                continue
            else:
                pass

        cc = 0
        while cc < 30:
            try:
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                unit = jsonData.split("<unit>")[m + cc].split("</unit>")[0]
                game = jsonData.split("<totalGameCount>")[m + cc].split("</totalGameCount>")[0]
                bb = jsonData.split("<bonusCount>")[m + cc].split("</bonusCount>")[0]
                rb = jsonData.split("<regularCount>")[m + cc].split("</regularCount>")[0]
                art = jsonData.split("<artCount>")[m + cc].split("</artCount>")[0]

                image = "https://m.site777.jp" + jsonData.split("<bigGraphUrl>")[m + cc].split("</bigGraphUrl>")[0].replace("amp;","")

                k = 0
                while(k < 10):
                    try:
                        filename =  'site777_2/dai_'+str(dn)+'.png'
                        download_img(image, filename)

                        time.sleep(wait)

                        Target_path = filename

                        template = cv2.imread(Template_s_path)
                        template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)

                        img_rgb = cv2.imread(Target_path)
                        img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)

                        res = cv2.matchTemplate(img_gray,template_gray,cv2.TM_CCOEFF_NORMED)

                        threshold = 0.6
                        loc = np.where( res >= threshold)

                        Mergin = 5
                        index = []
                        for iii in range(len(loc[1][:])):
                            count = 0
                            for j in range(len(loc[1][:])):
                                if abs(loc[1][:][iii] - loc[1][:][j]) < Mergin and abs(loc[0][:][iii] - loc[0][:][j]) < Mergin:
                                    if count > 0:
                                        index.append(j)
                                    count = count + 1

                        loc = np.delete(loc, index, axis=1)

                        w, h = template_gray.shape[::-1]

                        cv2.imwrite("site777_2/template_result.png",img_rgb)

                        img_bin = cv2.Canny(img_gray,128,200)
                        cv2.imwrite("site777_2/bin_result.png",img_bin)

                        count = 0

                        for x in range(0,234):
                            xx = 238 - x
                            a = 9
                            b = 442
                            if xx < 53:
                                a = 248
                                b = 435
                            for y in range(a,b):
                                if y < 332 or y > 334:
                                    if all(img_bin[y, xx]==[255, 255, 255]):
                                        point = 333 - y
                                        count = count + 1
                                        break
                            if count == 1:
                                break

                        samai = round(dot_point_s * point ,0)
                        if xx==10:
                            samai = 0

                        k = 10

                    except:
                        k = k + 1
                        continue

                    else:
                        pass

                temp = jsonData.split("<daiNo>")[m + cc + 1].split("</daiNo>")[0]
                li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb) + "|" + str(art)).split("|")
                print(li)
                if unit == "2173":
                    s20table.append(li)
                if unit == "561":
                    s5table.append(li)
                cc = cc + 1

            except:
                if cc == 0:
                    li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb) + "|" + str(art)).split("|")
                    print(li)
                    if unit == "2173":
                        s20table.append(li)
                    if unit == "561":
                        s5table.append(li)
                    roop = 1
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                cc = 30
                continue
            else:
                pass
            
    except:
        continue
    else:
        pass

wb.values_append("'P4RAW'!F" + str(int(p4row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": p4table})
wb.values_append("'P1RAW'!F" + str(int(p1row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": p1table})
wb.values_append("'P0.2RAW'!F" + str(int(p2row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": p2table})
wb.values_append("'S20RAW'!F" + str(int(s20row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": s20table})
wb.values_append("'S5RAW'!F" + str(int(s5row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": s5table})

['08/09', '025740', '1', '35385.0', '-20462.0', '11', '3']
['08/09', '025667', '2', '36154.0', '2107.0', '23', '7']
['08/09', '025572', '3', '23333.0', '-5416.0', '23', '11']
['08/09', '025107', '5', '44359.0', '-7823.0', '48', '15']
['08/09', '025593', '6', '31282.0', '-22569.0', '1', '1']
['08/09', '025777', '7', '43077.0', '-5115.0', '48', '16']
['08/09', '025850', '8', '42308.0', '-8124.0', '14', '6']
['08/09', '025573', '10', '27179.0', '-6619.0', '17', '7']
['08/09', '025647', '11', '42564.0', '1506.0', '68', '24']
['08/09', '025606', '12', '28718.0', '-16249.0', '6', '4']
['08/09', '025787', '13', '55385.0', '22570.0', '22', '18']
['08/09', '025838', '15', '27436.0', '-1805.0', '11', '4']
['08/09', '025579', '16', '40256.0', '-26781.0', '2', '2']
['08/09', '025603', '17', '32564.0', '-1805.0', '28', '14']
['08/09', '025521', '18', '29744.0', '-4814.0', '24', '11']
['08/09', '025521', '20', '43077.0', '2107.0', '47', '19']
['08/09', '025521', '21', '46923.0', '2709.0', '46', '21'

['08/09', '025844', '218', '0', '0', '0', '0']
['08/09', '025844', '220', '8462.0', '-5717.0', '5', '5']
['08/09', '025847', '221', '7179.0', '-5717.0', '4', '4']
['08/09', '025847', '222', '7179.0', '-6018.0', '3', '3']
['08/09', '025847', '223', '1538.0', '-1203.0', '2', '2']
['08/09', '025847', '225', '10256.0', '-8726.0', '3', '3']
['08/09', '025885', '226', '0', '0', '0', '0']
['08/09', '025868', '227', '5385.0', '1205.0', '8', '3']
['08/09', '025563', '228', '12051.0', '-5115.0', '8', '7']
['08/09', '025866', '230', '2051.0', '-1504.0', '2', '2']
['08/09', '025892', '231', '12564.0', '-5416.0', '7', '4']
['08/09', '025865', '232', '8205.0', '-3911.0', '10', '4']
['08/09', '025404', '233', '5897.0', '2408.0', '10', '2']
['08/09', '025835', '235', '16154.0', '-3610.0', '42', '9']
['08/09', '025575', '236', '12564.0', '5719.0', '27', '4']
['08/09', '025848', '237', '513.0', '1506.0', '4', '1']
['08/09', '025613', '238', '10000.0', '1205.0', '8', '1']
['08/09', '025874', '250', '8462

['08/09', '025523', '537', '17179.0', '15348.0', '41', '6']
['08/09', '025871', '538', '12308.0', '-6619.0', '5', '4']
['08/09', '025525', '550', '12308.0', '2408.0', '7', '1']
['08/09', '025683', '551', '20000.0', '-5416.0', '21', '7']
['08/09', '025773', '552', '0', '0', '0', '0']
['08/09', '025802', '553', '6410.0', '10533.0', '25', '4']
['08/09', '025533', '555', '20000.0', '7223.0', '35', '6']
['08/09', '025647', '556', '27436.0', '-2406.0', '47', '16']
['08/09', '025826', '557', '513.0', '-902.0', '0', '0']
['08/09', '025856', '558', '8462.0', '-6018.0', '1', '1']
['08/09', '025659', '560', '27949.0', '1806.0', '32', '16']
['08/09', '025651', '561', '5385.0', '-3911.0', '2', '2']
['08/09', '025824', '562', '15385.0', '5117.0', '24', '3']
['08/09', '025791', '563', '0', '0', '0', '0']
['08/09', '025667', '565', '11026.0', '-1504.0', '7', '5']
['08/09', '025665', '566', '11795.0', '16251.0', '17', '2']
['08/09', '025665', '567', '44615.0', '-22869.0', '11', '2']
['08/09', '025665',

['08/09', '025687', '765', '8974.0', '-6319.0', '1', '1']
['08/09', '025871', '766', '13846.0', '13242.0', '18', '5']
['08/09', '025718', '767', '4872.0', '-902.0', '22', '2']
['08/09', '025667', '768', '17949.0', '27686.0', '27', '3']
['08/09', '025544', '770', '7949.0', '-5416.0', '3', '3']
['08/09', '025615', '771', '14615.0', '3010.0', '31', '9']
['08/09', '025838', '772', '6923.0', '5719.0', '6', '1']
['08/09', '025285', '773', '5128.0', '-1504.0', '4', '1']
['08/09', '025523', '775', '16154.0', '-5717.0', '10', '6']
['08/09', '025483', '776', '5641.0', '302.0', '5', '1']
['08/09', '025790', '777', '7692.0', '-3610.0', '10', '4']
['08/09', '025501', '778', '9487.0', '904.0', '16', '6']
['08/09', '025712', '780', '17949.0', '11436.0', '72', '3']
['08/09', '025663', '781', '15641.0', '-1203.0', '12', '7']
['08/09', '025626', '782', '5385.0', '-1203.0', '6', '4']
['08/09', '025419', '783', '12308.0', '-3911.0', '10', '4']
['08/09', '025826', '785', '2564.0', '-1203.0', '1', '1']
['08

['08/09', '120128', '1207', '446', '278.0', '0', '0', '2']
['08/09', '120128', '1208', '718', '417.0', '0', '0', '3']
['08/09', '120128', '1210', '0', '0', '0', '0', '--']
['08/09', '120128', '1211', '1562', '-648.0', '0', '0', '4']
['08/09', '120128', '1212', '0', '0', '0', '0', '--']
['08/09', '120110', '1213', '2785', '-2269.0', '9', '2', '--']
['08/09', '120110', '1215', '2371', '694.0', '19', '6', '--']
['08/09', '120110', '1216', '5712', '-4167.0', '17', '11', '--']
['08/09', '120110', '1217', '3695', '972.0', '28', '9', '--']
['08/09', '120110', '1218', '3603', '-185.0', '21', '9', '--']
['08/09', '120110', '1220', '820', '370.0', '6', '4', '--']
['08/09', '120110', '1221', '4991', '-4583.0', '13', '5', '--']
['08/09', '120110', '1222', '6600', '-4954.0', '19', '11', '--']
['08/09', '120110', '1223', '4625', '-2685.0', '13', '19', '--']
['08/09', '120110', '1225', '1445', '-1019.0', '4', '4', '--']
['08/09', '120077', '1226', '53', '0', '0', '0', '--']
['08/09', '120077', '1227'

['08/09', '120026', '1515', '5417', '-417.0', '16', '16', '--']
['08/09', '120010', '1516', '4514', '-324.0', '14', '16', '--']
['08/09', '120010', '1517', '1942', '139.0', '9', '2', '--']
['08/09', '120010', '1518', '6193', '-1759.0', '14', '20', '--']
['08/09', '120010', '1520', '722', '-556.0', '1', '0', '--']
['08/09', '120010', '1521', '1245', '-556.0', '2', '4', '--']
['08/09', '120010', '1522', '2797', '417.0', '15', '4', '--']
['08/09', '120010', '1523', '2952', '-509.0', '9', '8', '--']
['08/09', '120010', '1525', '4433', '1944.0', '24', '14', '--']
['08/09', '120096', '1526', '2058', '-972.0', '5', '2', '--']
['08/09', '120096', '1527', '4942', '1157.0', '23', '14', '--']
['08/09', '120096', '1528', '3717', '-278.0', '13', '10', '--']
['08/09', '120096', '1530', '1840', '-741.0', '3', '7', '--']
['08/09', '119916', '1531', '1014', '694.0', '7', '1', '--']
['08/09', '119916', '1532', '741', '231.0', '4', '0', '--']
['08/09', '119916', '1533', '2347', '93.0', '9', '9', '--']
['

{'spreadsheetId': '1PG-sk-FpaxIWhexavaj2Jakkxb7vUd5CBfQLMtpnsT8',
 'updates': {'spreadsheetId': '1PG-sk-FpaxIWhexavaj2Jakkxb7vUd5CBfQLMtpnsT8',
  'updatedRange': 'S5RAW!F6400:M6439',
  'updatedRows': 40,
  'updatedColumns': 8,
  'updatedCells': 320}}